### Sending a request to the torchserve encoder service

In [ ]:
import httpx
import base64
from PIL import Image
from PIL.ImageOps import equalize, autocontrast
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
import json
encode_url="http://127.0.0.1:8080/predictions/sam_vit_h_encode" # make sure to select correct port. 70* for cpu, 80* for gpu
#encode_url = "https://segme-gpuel-ekfao79wi98g-617785108.us-east-1.elb.amazonaws.com/predictions/sam_vit_h_encode"
pth_fox = "../data/sample-img-fox.jpg"
input_point_fox = (150, 75)

pth_slick = "../data/tile_with_slick_512_512.png"
input_point_not_on_slick = (10, 120)
input_point_on_slick = (6, 120)
input_label = 1

In [ ]:
img_fox = Image.open(pth_fox)
img_fox

In [ ]:
Image.open("../data/wuPwKFUZ.jpeg").size

In [ ]:
img_slick = Image.open(pth_slick)
autocontrast(img_slick, cutoff=0, ignore=None, mask=None, preserve_tone=False)

In [ ]:
import io
def generate_dummy_array_and_encode(shape):
    # Create a dummy 3-channel numpy array with the given shape
    dummy_array = np.random.randint(0, 256, size=(shape[0], shape[1], 3), dtype=np.uint8)

    # Convert the numpy array to an image
    img = Image.fromarray(dummy_array)

    # Save the image to a buffer in PNG format
    buffer = io.BytesIO()
    img.save(buffer, format='PNG')
    buffer.seek(0)

    # Encode the buffer content as base64
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')

    # Store the encoded image in a dictionary
    result = {'encoded_image': encoded_image}

    return result

## Testing arbitrarily sized images, 2049 or greater should error 500 with detailed message.

In [ ]:
shape = (2049, 2049)
payload = generate_dummy_array_and_encode(shape)

In [ ]:
%%time
try:
    response = httpx.post(encode_url, json=payload, timeout=None)
except (BrokenPipeError, httpx.RemoteProtocolError) as e:
    print("wait and try again")

In [ ]:
# reads image as bytes, converts bytes to string so it can be sent as a post request
with open(pth_slick, 'rb') as f:
    byte_string = f.read()
    base64_string = base64.b64encode(byte_string).decode('utf-8')

payload = {"encoded_image": base64_string}

### Time to encode image on 1080 Ti GPU

In [ ]:
%%time
try:
    response = httpx.post(encode_url, json=payload, timeout=None)
except (BrokenPipeError, httpx.RemoteProtocolError) as e:
    print("wait and try again")

In [ ]:
response.content

In [ ]:
response

### Image embeddings for the image above

In [ ]:
encoded_embedding_string = response.json()['image_embedding']
base64_bytes = base64.b64decode(encoded_embedding_string)
image_embedding = np.frombuffer(base64_bytes, dtype=np.float32)
image_embedding

### Sending the image embeddings to the decoder service

In [ ]:
img_shape = np.array(img_slick).shape
decode_payload = {
    "image_embeddings": encoded_embedding_string,
    "image_shape": img_shape,
    "input_label": input_label,
    "input_point": input_point_on_slick
}

In [ ]:
%%time
decode_url="http://127.0.0.1:8080/predictions/sam_vit_h_decode" # make sure to select correct port. 70* for cpu, 80* for gpu
#decode_url="https://sas.ds.io/predictions/sam_vit_h_decode"
response = httpx.post(decode_url, json=decode_payload, timeout=None)

In [ ]:
response

In [ ]:
encoded_masks_string = response.json()['masks']
base64_bytes_masks = base64.b64decode(encoded_masks_string)
masks = np.frombuffer(base64_bytes_masks, dtype=np.float32)

#### We reshape the masks to the original image shape, adding a channel for the alpha channel, so it is 4 instead of 3

In [ ]:
masks = masks.reshape((1, 4, 512, 512))
masks = masks > .5

### With our encoder and decoder service, we get a solid mask prediction by just supplying a point on the object of interest!

In [ ]:
def show_mask(mask, ax):
    color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.array(img_slick))
show_mask(masks[:,0,:,:], plt.gca())
input_point_arr = np.array(input_point_on_slick)[np.newaxis,:]
input_label_arr = np.array(input_label)[np.newaxis]
show_points(input_point_arr, input_label_arr, plt.gca())
plt.axis('off')
plt.show() 